In [ ]:
from sfcn import SFCN
import numpy as np
import pandas as pd
from volumedatagenerator import VolumeDataGeneratorRegression
import matplotlib.pyplot as plt

In [ ]:
# col_list = pd.read_csv('fields/subcortical_volumes.csv')['field id'].to_list()

# columns=['path']
# for col in col_list:
#     columns.append(str(col)+'-2.0')

In [ ]:
train_df = pd.read_csv('csv/split_train.csv', index_col='eid').dropna()
valid_df = pd.read_csv('csv/split_valid.csv', index_col='eid').dropna()
test_df = pd.read_csv('csv/split_test.csv', index_col='eid').dropna()

In [ ]:
name = 'sfcn_avg_norm'
batch_size = 5
gpu_num = 5
cpu_workers = 5
epochs_num = 64

index=0

In [ ]:
input_dim = [182, 218, 182]
num_output = len(train_df.columns)-1

In [ ]:
train_gen = VolumeDataGeneratorRegression(
    sample_df=train_df, 
    batch_size=batch_size, 
    #num_reg_classes=num_output, 
    dim=input_dim,
    output_preprocessing='quantile')

scaler_instance = train_gen.get_scaler_instance()

In [ ]:
valid_gen = VolumeDataGeneratorRegression(
    sample_df=valid_df, 
    batch_size=batch_size, 
    #num_reg_classes=num_output, 
    dim=input_dim,
    output_scaler=scaler_instance,
    shuffle=False)

In [ ]:
test_gen = VolumeDataGeneratorRegression(
    sample_df=test_df, 
    batch_size=batch_size, 
    #num_reg_classes=num_output, 
    dim=input_dim,
    output_scaler=scaler_instance,
    shuffle=False
)

In [11]:
model = SFCN(
        input_dim=[182, 218, 182, 1], 
        output_dim=num_output,
        conv_num_filters=[32, 64, 128, 256, 256, 64], 
        conv_kernel_sizes=[3, 3, 3, 3, 3, 1], 
        conv_strides=[1, 1, 1, 1, 1, 1],
        conv_padding=['same', 'same', 'same', 'same', 'same', 'valid'],
        pooling_size=[2, 2, 2, 2, 2],
        pooling_type=['avg_norm', 'avg_norm', 'avg_norm', 'avg_norm', 'avg_norm'],
        batch_norm=True,
        dropout=False,
        softmax=False,
        gpu_num=gpu_num,
        name=name+'_'+str(index)
        )
model.compile(learning_rate=3e-4)

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [ ]:
model.train_generator(train_gen, valid_gen, batch_size=batch_size, epochs=epochs_num, workers=cpu_workers)

In [12]:
model.load_weights('weights/checkpoint_' + name + '_' + str(index))
model.evaluate_generator(valid_gen, batch_size, filename=name + '_val', workers=cpu_workers)
model.evaluate_generator(test_gen, batch_size, filename=name + '_test', workers=cpu_workers)

2021-12-17 09:15:55.096748: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:656] In AUTO-mode, and switching to DATA-based sharding, instead of FILE-based sharding as we cannot find appropriate reader dataset op(s) to shard. Error: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_2"
op: "FlatMapDataset"
input: "TensorDataset/_1"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_flat_map_fn_1738"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  va

KeyboardInterrupt: 

# AVG pool

In [ ]:
# name = 'sfcn_avg_pool'

In [ ]:
# model = SFCN(
#         input_dim=[182, 218, 182, 1], 
#         output_dim=num_output,
#         conv_num_filters=[32, 64, 128, 256, 256, 64], 
#         conv_kernel_sizes=[3, 3, 3, 3, 3, 1], 
#         conv_strides=[1, 1, 1, 1, 1, 1],
#         conv_padding=['same', 'same', 'same', 'same', 'same', 'valid'],
#         pooling_size=[2, 2, 2, 2, 2],
#         pooling_type=['avg_pool', 'avg_pool', 'avg_pool', 'avg_pool', 'avg_pool'],
#         batch_norm=True,
#         dropout=False,
#         softmax=False,
#         gpu_num=gpu_num,
#         name=name+'_'+str(0)
#         )
# model.compile(learning_rate=3e-4)

In [ ]:
# model.train_generator(train_gen, valid_gen, batch_size=batch_size, epochs=40, workers=cpu_workers)

In [ ]:
# model.load_weights('weights/checkpoint_' + name + '_' + str(0))
# model.evaluate_generator(valid_gen, batch_size, filename=name + '_val', workers=cpu_workers)
# model.evaluate_generator(test_gen, batch_size, filename=name + '_test', workers=cpu_workers)

In [ ]:
#model.train_generator(train_gen, valid_gen, batch_size=batch_size, epochs=40)

In [ ]:
#model.load_weights('weights/checkpoint_' + name)
#model.evaluate_generator(valid_gen, batch_size)

In [ ]:
# from pathlib import Path
# from datetime import date

# r2 = -0.29363421770141684
# mae = 0.27212154832328556
# mse = 0.10790403615065694
# today = date.today()

# columns = ['date','r2','mae','mse']

# # df = pd.DataFrame(entry, columns=columns, index='date')

# filedir = Path.cwd().joinpath('results')
# filedir.mkdir(parents=True, exist_ok=True)

# fn = filedir.joinpath(name +'.csv')


# if fn.exists():
#     entry ={
#         'date':today,
#         'r2':r2,
#         'mae':mae,
#         'mse':mse }
#     df = pd.read_csv(fn)    
#     df = df.append(entry, ignore_index=True)
# else:
#     df = pd.DataFrame([[today, r2, mae, mse]], columns=columns)

# df.to_csv(fn, index=False)

In [ ]:
#model.model.evaluate(valid_gen)
# train_df['path'].iloc[0]aa

In [ ]:

#data_size=200
#X = np.random.normal(loc=5, scale=5, size=(data_size, *(input_dim), 1))
#y = np.random.normal(loc=1, scale=1, size=(data_size, num_output))
#model.train(X, y, batch_size, 300)

In [ ]:
#x_test = np.random.normal(loc=8, scale=3, size=(1, 160, 192, 160, 1))

In [ ]:
#model.predict(x_test)

In [ ]:
#y_test = np.mean(x_test, axis=(1,2,3,4))
#print(y_test)